#Installing Necessities

In [19]:
# run this cell, then restart the runtime before continuing
!pip install nlp
!pip install transformers

In [20]:
import tensorflow as tf
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [21]:
import numpy as np
import pandas as pd
import random
import requests

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import seaborn as sns

from tqdm.notebook import tqdm

from transformers import BertModel, BertTokenizer
from transformers import AdamW,get_linear_schedule_with_warmup

from sklearn.metrics import recall_score
from sklearn.metrics import f1_score 

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import RandomSampler,SequentialSampler,DataLoader

import torch
import torch.nn as nn
from transformers import BertModel

import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
############################ SENTIMENT ANALYSIS #################################################
SENTIMENT_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt'
SENTIMENT_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_text.txt'
SENTIMENT_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt'

SENTIMENT_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt'
SENTIMENT_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_labels.txt'
SENTIMENT_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt'

############################ HATE #################################################
HATE_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt'
HATE_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_text.txt'
HATE_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt'

HATE_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt'
HATE_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_labels.txt'
HATE_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt'

############################ OFFENSIVE LANGUAGE#################################################
OFFENSE_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt'
OFFENSE_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt'
OFFENSE_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt'

OFFENSE_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt'
OFFENSE_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt'
OFFENSE_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt'

############################ IRONY#################################################
IRONY_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_text.txt'
IRONY_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_text.txt'
IRONY_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_text.txt'

IRONY_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_labels.txt'
IRONY_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_labels.txt'
IRONY_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_labels.txt'

############################ EMOTION#################################################
EMOTION_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt'
EMOTION_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt'
EMOTION_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt'

EMOTION_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt'
EMOTION_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt'
EMOTION_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt'

In [23]:
def preprocess(df): 
    lemmatizer  = WordNetLemmatizer()
    ignore_words = ['user', 'st'] 
    df['processed_tweets'] = df['tweet'].replace('[^a-zA-Z]',' ', regex=True,
                                                  inplace=False)
    df['processed_tweets'] = df['processed_tweets'].apply(lambda x: [w.lower() for w in x.split()])
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ([word for word in tweet if not word in stopwords.words("english")]))
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ([lemmatizer.lemmatize(word) for word in tweet]))

    df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ' '.join([word for word in tweet if len(word)>2]))

    df['processed_tweets'] = df['processed_tweets'].apply(lambda x: ' '.join([word for word in x.split() if not word in ignore_words]))
    
    df["sentence_length"] = df.tweet.apply(lambda x: len(str(x).split()))
    return df


# Wrapper to convert text data to pandas Dataframe
def txt_to_df(data, label, classification_task):
    tweet = []
    sentiments = []
    for sentence in data.split('\n'):
        tweet.append(sentence)
    for sentiment in label.split('\n'):
        try:
            sentiments.append(int(sentiment))
        except ValueError:
            pass
    df= pd.DataFrame(tweet[:-1], columns=['tweet'])
    df['label'] = sentiments
    if classification_task == 'Sentiment_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Negative'if x==0 else ('Neutral' if x==1 else 'Positive'))
    if classification_task == 'hate_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Not-hate'if x==0 else 'hate')
    if classification_task == 'offensive_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Not-offensive 'if x==0 else 'offensive')
    if classification_task == 'irony':
        df['sentiment'] = df.label.apply(lambda x: 'Not-irony 'if x==0 else 'irony')
    if classification_task == 'emotion':
        df['sentiment'] = df.label.apply(lambda x: 'Anger'if x==0 else ('Joy' if x==1 else ('Optimism' if x==2 else 'Sadness')))      
    return df


def prepare_dataset(TRAIN_TEXT, TRAIN_LABEL, VAL_TEXT, VAL_LABEL, TEST_TEXT, TEST_LABEL, classification_task):
  # Reading Train, Vvalidation & Test data from tweeteval Github Repo.
  train_tweets_txt = requests.get(TRAIN_TEXT).text
  train_labels_txt = requests.get(TRAIN_LABEL).text

  val_tweets_txt = requests.get(VAL_TEXT).text
  val_labels_txt = requests.get(VAL_LABEL).text

  test_tweets_txt = requests.get(TEST_TEXT).text
  test_labels_txt = requests.get(TEST_LABEL).text

  # Converting text data to pandas Dataframe
  train_df = txt_to_df(train_tweets_txt, train_labels_txt, classification_task)
  val_df = txt_to_df(val_tweets_txt, val_labels_txt, classification_task)
  test_df = txt_to_df(test_tweets_txt, test_labels_txt, classification_task)

  train_df = preprocess(train_df)
  val_df = preprocess(val_df)
  test_df = preprocess(test_df)  

  return train_df, val_df, test_df



In [24]:
def initialize_bert(num_of_class):
  model = BertModel.from_pretrained('bert-base-uncased')

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
  return model, tokenizer

In [25]:
def encode_data(tokenizer, df, max_sequence_length=64):
  input_ids = []
  attention_masks = []
  for sent in df.processed_tweets.values:
    encoder = tokenizer.encode_plus(
              text=sent,  # Preprocess sentence
              add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
              max_length=max_sequence_length, # Max length to truncate/pad
              pad_to_max_length=True,         # Pad sentence to max length
              #return_tensors='pt',           # Return PyTorch tensor
              return_attention_mask=True,      # Return attention mask
              truncation = True
              )
    input_ids.append(encoder.get('input_ids'))
    attention_masks.append(encoder.get('attention_mask'))


  # Convert lists to tensors
  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)
  labels = torch.tensor(df.label.values)

  return input_ids, attention_masks, labels

In [26]:
def get_tesnsor_dataset(input_ids, attention_masks, labels):
  return TensorDataset(input_ids, attention_masks, labels)


In [27]:
def dataloader_object(data, batch_size=16):
  dataloader = DataLoader(
    data,
    sampler= RandomSampler(data),
    batch_size = batch_size)
  return dataloader

In [28]:
# Get all of the model's parameters as a list of tuples.

def print_model_params(model):
  params = list(model.named_parameters())
  print('The BERT model has {:} different named parameters.\n'.format(len(params)))
  print('==== Embedding Layer ====\n')
  for p in params[0:5]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
  print('\n==== First Transformer ====\n')
  for p in params[5:21]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
  print('\n==== Output Layer ====\n')
  for p in params[-4:]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [29]:
def calc_max_len(tokenizer, df_train, df_test):  
  # Concatenate train data and test data
  processed_tweets = np.concatenate([df_train.processed_tweets.values, df_test.processed_tweets.values])

  # Encode our concatenated data
  encoded_tweets = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in processed_tweets]

  # Find the maximum length
  max_len = max([len(sentence) for sentence in encoded_tweets])
  print('Max length: ', max_len)
  return max_len

In [30]:
def f1_score_func(predictions,y_labelled):
    preds_flatten = np.argmax(predictions,axis=1).flatten()
    labels_flatten = y_labelled.flatten()
    return f1_score(labels_flatten,preds_flatten,average = 'macro')

In [31]:
def recall_score_func(predictions,y_labelled):
    preds_flatten = np.argmax(predictions,axis=1).flatten()
    labels_flatten = y_labelled.flatten()
    return recall_score(labels_flatten,preds_flatten,average = 'macro')

In [32]:
def load_model_to_device(bert_model):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  bert_model.to(device)
  print(f"Loading:{device}")
  return device

In [33]:
def evaluate(bert_model, device, dataloader_val):
    bert_model.eval()
    
    val_loss = []
    val_accuracy = []
    
    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':  batch[0],
                  'attention_mask':batch[1],
                  'labels': batch[2]
                 }
        with torch.no_grad():
            logits = bert_model(inputs['input_ids'], inputs['attention_mask'])
          
        loss =  loss_fn(logits, inputs['labels'] )
        val_loss.append(loss.item())
        
  
        predictions = torch.argmax(logits, dim=1).flatten()
        ground_truth = inputs['labels']
        
        accuracy = f1_score(ground_truth.tolist(), predictions.tolist(), average='macro')

        val_accuracy.append(accuracy)

    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
 
    return val_loss,val_accuracy

In [34]:
def evaluate_wrapper(bert_model, device, dataloader_test):
  val_loss, val_accuracy = evaluate(bert_model,device, dataloader_test)

  # if classification_task == 'SENTIMENT_ANALYSIS':
  #   test_score = recall_score_func(predictions,true_vals)
  # else:
  #   test_score = f1_score_func(predictions,true_vals)  

  tqdm.write(f'Val Loss:{val_loss}\nTest Score:{val_accuracy}')

In [35]:
# For the purposes of fine-tuning, the authors recommend choosing from the following values:
# Batch size: 16, 32 (We chose 32 when creating our DataLoaders).
# Learning rate (Adam): 5e-5, 3e-5, 2e-5 (We’ll use 2e-5).
# Number of epochs: 2, 3, 4 (We’ll use 4).

def initialize_optimizer(model, freeze_bert, dataloader, lr=1e-5, epochs=2):
  classifier = BertClassifier(model, freeze_bert)
  optimizer = AdamW(classifier.parameters(),lr,eps = 1e-8)

  scheduler = get_linear_schedule_with_warmup(
              optimizer,
      num_warmup_steps = 0,
    num_training_steps = len(dataloader)*epochs 
  )
  return optimizer, scheduler, classifier

In [36]:
loss_fn = nn.CrossEntropyLoss()

def init_training(bert_model, optimizer, scheduler, epochs, device, dataloader_train, dataloader_val): 
  for epoch in tqdm(range(1,epochs+1)):
      bert_model.train()
      
      loss_train_total=0
      
      progress_bar = tqdm(dataloader_train,desc = "Epoch: {:1d}".format(epoch),leave = False,disable = False)
      
      
      for batch in progress_bar:
          bert_model.zero_grad()
          
          batch = tuple(b.to(device) for b in batch)
          
          inputs = {
              "input_ids":batch[0],
              "attention_mask":batch[1],
              "labels":batch[2]
              
          }
          logits = bert_model(inputs['input_ids'], inputs['attention_mask'])
          
          loss =  loss_fn(logits, inputs['labels'] )
          loss_train_total +=loss.item()
          loss.backward()
          
          torch.nn.utils.clip_grad_norm(bert_model.parameters(),1.0)
          
          optimizer.step()
          scheduler.step()
          
          
          progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
      
      tqdm.write('\nEpoch {epoch}')
      
      loss_train_avg = loss_train_total/len(dataloader_train)
      tqdm.write(f'Training Loss: {loss_train_avg}')
      val_loss, val_accuracy = evaluate(bert_model,device, dataloader_val)

      # if classification_task == 'SENTIMENT_ANALYSIS':
      #   test_score = recall_score_func(predictions,true_vals)
      # else:
      #   test_score = f1_score_func(predictions,true_vals)
      

      tqdm.write(f'Val Loss:{val_loss}\n Val Score:{val_accuracy}')
      

In [56]:
def fineTune_bert(batch_size, lr, epochs, freeze_bert):

  num_of_class= len(train_df.sentiment.unique())

  model, tokenizer = initialize_bert(num_of_class)
  max_length = calc_max_len(tokenizer, train_df, test_df) 

  input_ids_train, attention_masks_train, labels_train = encode_data(tokenizer, train_df, max_length)
  input_ids_eval, attention_masks_eval, labels_eval = encode_data(tokenizer, val_df, max_length)
  input_ids_test, attention_masks_test, labels_test =  encode_data(tokenizer, test_df, max_length)

  data_train = get_tesnsor_dataset(input_ids_train,attention_masks_train,labels_train)
  data_eval = get_tesnsor_dataset(input_ids_eval,attention_masks_eval,labels_eval)
  data_test = get_tesnsor_dataset(input_ids_test,attention_masks_test,labels_test)

  dataloader_train = dataloader_object(data_train, batch_size) 
  dataloader_eval = dataloader_object(data_eval, batch_size) 
  dataloader_test = dataloader_object(data_test, batch_size)

  optimizer, scheduler, classifier = initialize_optimizer(model, freeze_bert, dataloader_train, lr, epochs)
  print_model_params(classifier)

  device = load_model_to_device(classifier)

  init_training(classifier,optimizer,  scheduler, epochs, device, dataloader_train, dataloader_test)
  evaluate_wrapper(classifier, device, dataloader_test)

In [70]:


# Create the BertClassfier class
class BertClassifier(nn.Module):
    def __init__(self, bert_model, freeze_bert):
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        input_size, hidden_size, output_size = 768, 50, 2

        # Instantiate BERT model
        self.bert = bert_model

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
            # nn.Dropout(0.2),
            nn.Linear(hidden_size, output_size),
            )


        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

        # for i,param in enumerate(self.bert.embeddings.parameters()):
        #     param.requires_grad = False
        # # #     print(i)

        #   # if freeze_layer_count == -1, we only freeze the embedding layer
        #   # otherwise we freeze the first `freeze_layer_count` encoder layers
        # for layer in self.bert.encoder.layer[:-4]:
        #     for param in layer.parameters():
        #         param.requires_grad = False



        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask) 
        last_hidden_state_cls = outputs[0][:, 0, :]
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [71]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
import torch

classification_task_dict = {'SENTIMENT_ANALYSIS' : 'Sentiment_analysis',
                      'HATE_ANALYSIS' : 'hate_analysis',
                      'OFFENSIVE_LANGUAGE' : 'offensive_analysis',
                      'IRONY' : 'irony',
                      'EMOTION' : 'emotion'
                      }
class_dict = {'SENTIMENT_ANALYSIS' :['Negative', 'Neutral', 'Positive'],
              'HATE_ANALYSIS' : ['Not-hate', 'hate'],
              'OFFENSIVE_LANGUAGE' : ['Not-offensive', 'offensive'],
              'IRONY' : ['non-irony', 'irony'],
              'EMOTION' : ['anger', 'joy', 'optimism', 'sadness']}

config = {'batch_size' : 21,
          'lr' : 5e-5,
          'epochs' : 3,
          'freeze_bert' : True
        }

for classification_task, task in classification_task_dict.items():
  torch.cuda.empty_cache()
  print('=========================================')
  print('CLASSIFICATION TASK: {}'.format(classification_task))
  print('=========================================')
  if classification_task == 'SENTIMENT_ANALYSIS':
    continue
    train_df, val_df, test_df = prepare_dataset(SENTIMENT_TRAIN_TEXT, SENTIMENT_TRAIN_LABEL,
                        SENTIMENT_VALIDATION_TEXT, SENTIMENT_VALIDATION_LABEL,
                        SENTIMENT_TEST_TEXT, SENTIMENT_TEST_LABEL, classification_task_dict['SENTIMENT_ANALYSIS']
                        )

  if classification_task == 'HATE_ANALYSIS':
    # continue
    train_df, val_df, test_df = prepare_dataset(HATE_TRAIN_TEXT, HATE_TRAIN_LABEL,
                        HATE_VALIDATION_TEXT, HATE_VALIDATION_LABEL,
                        HATE_TEST_TEXT, HATE_TEST_LABEL, classification_task_dict['HATE_ANALYSIS']
                        )
    
  if classification_task == 'OFFENSIVE_LANGUAGE':
    continue
    train_df, val_df, test_df = prepare_dataset(OFFENSE_TRAIN_TEXT, OFFENSE_TRAIN_LABEL,
                        OFFENSE_VALIDATION_TEXT, OFFENSE_VALIDATION_LABEL,
                        OFFENSE_TEST_TEXT, OFFENSE_TEST_LABEL, classification_task_dict['OFFENSIVE_LANGUAGE']
                        )
    
  if classification_task == 'IRONY':
    continue
    train_df, val_df, test_df = prepare_dataset(IRONY_TRAIN_TEXT, IRONY_TRAIN_LABEL,
                        IRONY_VALIDATION_TEXT, IRONY_VALIDATION_LABEL,
                        IRONY_TEST_TEXT, IRONY_TEST_LABEL, classification_task_dict['IRONY']
                        )
  if classification_task == 'EMOTION':
    continue
    train_df, val_df, test_df = prepare_dataset(EMOTION_TRAIN_TEXT, EMOTION_TRAIN_LABEL,
                        EMOTION_VALIDATION_TEXT, EMOTION_VALIDATION_LABEL,
                        EMOTION_TEST_TEXT, EMOTION_TEST_LABEL, classification_task_dict['EMOTION']
                        )    
  frames = [train_df, val_df]    
  train_df = pd.concat(frames)
  train_df.reset_index(inplace=True)

  fineTune_bert(config['batch_size'], config['lr'], config['epochs'], config['freeze_bert'])


CLASSIFICATION TASK: SENTIMENT_ANALYSIS
CLASSIFICATION TASK: HATE_ANALYSIS
Max length:  82
The BERT model has 203 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.den


Epoch {epoch}
Training Loss: 0.6638033341311809



Val Loss:0.6702482293189411
 Val Score:0.5280623634900313



Epoch {epoch}
Training Loss: 0.6408766486859672



Val Loss:0.6692575044195417
 Val Score:0.5919156057894928



Epoch {epoch}
Training Loss: 0.6317118295083756



Val Loss:0.6686542512665332
 Val Score:0.5994721887807579




Val Loss:0.6686793255134368
Test Score:0.600651084121013
CLASSIFICATION TASK: OFFENSIVE_LANGUAGE
CLASSIFICATION TASK: IRONY
CLASSIFICATION TASK: EMOTION
